In [1]:
import numpy as np

data = np.load("./../data/collected_data.npy", allow_pickle=True)


In [2]:
import torch
print(torch.cuda.device_count())


1


In [ ]:
len(data)

In [ ]:
data[0].shape

In [ ]:
data[0][0]

In [ ]:
# Take first episode's data
episode_data = data[0]

# Extract the states (images)
states = [x[0] for x in episode_data]

# Take some samples (e.g., first 5)
sample_states = states[0:999]


In [ ]:
import matplotlib.pyplot as plt

cols = 40
for idx, image in enumerate(sample_states):
    # plot with 10 columns and number of rows based on number of samples
    plt.subplot(int(np.ceil(len(sample_states)/cols)), cols, idx+1)
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    # add some vertical space between the images
    plt.subplots_adjust(hspace=0.6)
    # label the axes with the action
    plt.title(np.round(episode_data[idx][1],2), fontsize=3)
    #make the length of the figure relative to the number of samples
    plt.gcf().set_size_inches(20, len(sample_states)/cols)
    

plt.show()


In [ ]:
import pandas as pd

flattened_data = []
episode_count = 1

for episode in data:
    for step in episode:
        # Convert step to list and prepend episode_count
        step_list = [episode_count] + list(step)
        flattened_data.append(step_list)
    episode_count += 1

df = pd.DataFrame(
    flattened_data,
    columns=["Episode", "State", "Action", "Reward", "Done", "Truncated"]
)




In [ ]:
print(df.head())
print(df.describe())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(df['Reward'].values, bins=100, kde=True)
plt.show()


In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from ..src.worldmodels1.cnnvae import VAE

ModuleNotFoundError: No module named 'worldmodels1'

In [ ]:
vae = VAE()